<h1> Learning jraph </h1>
<h2> All code copied from https://github.com/deepmind/jraph
    <b>Do not track this file, not mine to upload</b>
    </h2>
  

In [13]:
import jraph
import jax.numpy as jnp

ModuleNotFoundError: No module named 'jraph.models'

In [4]:
# Define a three node graph, each node has an integer as its feature.
node_features = jnp.array([[0.],[1.],[2.]])

# We will construct a graph for which there is a directed edge between each node
# and its successor. We define this with `senders` (source nodes) and `receivers`
# (destination nodes).
senders = jnp.array([0,1,2])
receivers = jnp.array([1,2,3])

# You can optionally add edge attributes.
edges = jnp.array([[5.], [6.], [7.]]) ##Edge weights? Will be useful when definiing Francois' weight assignment

# We then save the number of nodes and the number of edges.
# This information is used to make running GNNs over multiple graphs
# in a GraphsTuple possible.

n_node = jnp.array([3])
n_edge = jnp.array([3])

# Optionally you can add `global` information, such as a graph label.

global_context = jnp.array([[1]]) # Same feature dimensions as nodes and edges.
graph = jraph.GraphsTuple(nodes  = node_features, senders = senders, receivers = receivers,
                         edges = edges, n_node = n_node, n_edge = n_edge, globals = global_context)



In [29]:
edges

DeviceArray([[5.],
             [6.],
             [7.]], dtype=float32)

In [5]:
#A GraphsTuple can have more than one graph.
two_graph_graphstuple = jraph.batch([graph,graph])

In [6]:
#The node and edge features are stacked on the leading axis.
jraph.batch([graph,graph]).nodes

DeviceArray([[0.],
             [1.],
             [2.],
             [0.],
             [1.],
             [2.]], dtype=float32)

In [8]:
#You can tell which nodes are from which graph by looking at n_node.
jraph.batch([graph,graph]).n_node

DeviceArray([3, 3], dtype=int32)

In [9]:
node_targets = jnp.array([[True],[False],[True]])
graph = graph._replace(nodes ={'inputs' : graph.nodes, 'targets' : node_targets})

<h2> Model Zoo
    </h2>

In [10]:
# As one example, we just pass the edge features straight through.
def update_edge_fn(edge, sender, receiver, globals_):
    return edge

In [24]:
@jraph.concatenated_args
def update_edge_fn(concatenated_features):
    return concatenated_features

def update_node_fn(nodes, sent_attributes, received_attributes, global_attributes):
    return nodes

def update_global_fn(concatenated_features):
    return concatenated_features

In [25]:
net = jraph.GraphNetwork(update_edge_fn=update_edge_fn,
                        update_node_fn=update_node_fn,
                        update_global_fn=update_global_fn)

In [26]:
updated_graph = net(graph)

TypeError: add does not accept dtype bool at position 0. Accepted dtypes at position 0 are subtypes of floating, complexfloating, integer.